In [1]:
import glob
import nltk
import os
import pandas as pd
import numpy as np
import re
import string
import fileinput

from tqdm.notebook import tqdm
from lxml import etree

TOKENIZER = nltk.tokenize.word_tokenize

In [23]:
parser = etree.XMLParser()
punctuation = "[{`,.?!:;/\()''""}¬]" #eventueel hier de hyphen weghalen zodat deze later gemerged kunnen worden in het outputfile

all_chronicles_xml = glob.glob(os.path.join('../../path-to-baseline_corpus/xml/*.xml'))
rows = []

for file in all_chronicles_xml:
    xml = etree.parse(file)

    for elem in xml.getiterator():
        elem.tag = etree.QName(elem).localname
    etree.cleanup_namespaces(xml)
    
    d_dict = {}
    tokens = []
    
    line_elements = xml.xpath('//l')
    for i, line in tqdm(enumerate(line_elements),
                        total=len(line_elements)):
        text =  ' '.join(line.xpath('descendant::text()'))
        tokens.extend(TOKENIZER(text))
        
    prev_tok = None
    all_tokens = []
    for token in tokens:
        if '¬' in token:
            prev_tok = token
            continue
        elif prev_tok is not None:
            token = prev_tok.replace('¬', '') + token 
            prev_tok = None
        remove_punct = ''.join([c for c in token if c not in punctuation])
        if len(remove_punct) > 0:
            all_tokens.append(token)
    d_dict['filename'] = re.search(r"(?<=xml/)(.*?)(?=.xml)", file).group(0)
    #d_dict['filename'] = re.search(r"(?<=test/)(.*?)(?=.xml)", file).group(0)        
    d_dict['tokens'] = len(all_tokens)
    rows.append(d_dict)

In [31]:
re.search(r"(?<=test/)(.*?)(?=.xml)", file).group(0)

'1748_Berg_Folk'

In [24]:
pd.DataFrame(rows)

,filename,tokens
0,1789_Doet_Duys,5337
1,1672_Amst_Wate,17674
2,1574_Antw_EykP,69571
3,1668_Gent_Bill_08,69732
4,1791_Purm_Louw,266020
...,...,...
101,1805_Brus_Goet,80403
102,1668_Gent_Bill_07,72110
103,1681_Zwol_Catt,60354
104,1671_Rott_Lois,92492


In [25]:
df_xml = pd.DataFrame(rows)

In [26]:
df_xml.loc[df_xml['filename'] == '1604_Bosc_Anon']

,filename,tokens
94,1604_Bosc_Anon,37549


In [27]:
all_chronicles_xlsx = glob.glob(os.path.join('../corpus/corpus_191121/baseline_corpus/xlsx/*.xlsx'))
rows_xlsx = []

for file in all_chronicles_xlsx:
    w_dict = {}
    c_dict = {}
    c_dict['filename'] = re.search(r"(?<=xlsx/)(.*?)(?=.xlsx)", file).group(0)
    try:
        informatiebron = pd.read_excel(file, sheet_name='informatiebron')
        c_dict['informatiebron'] = int(len(informatiebron))
    except:
        pass
    try:
        ontvanger = pd.read_excel(file, sheet_name='ontvanger')
        c_dict['ontvanger'] = int(len(ontvanger))
    except:
        pass
    try:
        waarneming = pd.read_excel(file, sheet_name='waarneming')
        c_dict['waarneming'] = int(len(waarneming))
        w_dict = waarneming.groupby('waarneming')['waarneming'].count().to_dict()
        c_dict.update(w_dict)
    except:
        pass
    rows_xlsx.append(c_dict)

df_xlsx = pd.DataFrame(rows_xlsx)

In [28]:
df_xlsx

,filename,informatiebron,ontvanger,waarneming,Anders,Geschreven / Gelezen,Gezien,Mondeling / Gehoord
0,1758_Maas_anon,109.0,32.0,67.0,1.0,6.0,15.0,44.0
1,1643_Antw_anon,13.0,2.0,21.0,2.0,4.0,11.0,4.0
2,1846_Antw_Coni_01,90.0,12.0,32.0,NaN,20.0,4.0,8.0
3,1814_Worm_Ding,23.0,1.0,4.0,NaN,2.0,1.0,1.0
4,1671_Rott_Lois,79.0,9.0,11.0,NaN,5.0,3.0,3.0
...,...,...,...,...,...,...,...,...
101,1835_Gent_Call_03,26.0,24.0,35.0,4.0,13.0,1.0,17.0
102,1846_Antw_Coni_02,122.0,9.0,31.0,NaN,7.0,2.0,22.0
103,1796_Antw_Aars,21.0,2.0,2.0,NaN,NaN,2.0,NaN
104,1801_Huis_Anon,1.0,1.0,1.0,NaN,NaN,NaN,1.0


In [11]:
df_xlsx.loc[df_xlsx['filename'] == '1632_Amer_Anon']

,filename,informatiebron,ontvanger,waarneming,Anders,Geschreven / Gelezen,Gezien,Mondeling / Gehoord
79,1632_Amer_Anon,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_total = df_xml.merge(df_xlsx, on='filename', how ='inner')

In [30]:
df_total = df_total.fillna(0)

In [31]:
df_total.shape

(106, 9)

In [15]:
df_total.head()

,filename,tokens,informatiebron,ontvanger,waarneming,Anders,Geschreven / Gelezen,Gezien,Mondeling / Gehoord
0,1789_Doet_Duys,5337,4.0,2.0,3.0,0.0,0.0,1.0,2.0
1,1672_Amst_Wate,17674,16.0,3.0,2.0,1.0,1.0,0.0,0.0
2,1574_Antw_EykP,69571,11.0,13.0,15.0,0.0,0.0,4.0,11.0
3,1668_Gent_Bill_08,69732,101.0,30.0,74.0,4.0,12.0,12.0,46.0
4,1791_Purm_Louw,266020,476.0,133.0,268.0,1.0,43.0,110.0,114.0


In [32]:
df_total['written'] = df_total["Geschreven / Gelezen"]
df_total['heard'] = df_total['Mondeling / Gehoord']
df_total['seen'] = df_total['Gezien']
df_total['else'] = df_total['Anders']
df_total = df_total.drop(columns=["Geschreven / Gelezen", 'Mondeling / Gehoord', 'Gezien', 'Anders'])

In [33]:
corpus_metadata = '/Users/alielassche/surfdrive/Thesis_Alie/Sources/Chronicles_source_tagging.xlsx'

corpus = pd.read_excel(corpus_metadata, sheet_name='Corpus_najaar_2021_2')
merged_metadata = corpus.merge(df_total, left_on='Call_Number', right_on='filename', how ='left')

In [34]:
merged_metadata.head(20)

,Batch,Call_Number,Title,n-scans,Gecontroleerd in project 2,Subcorpus,scans-gelabeld,filename,tokens,informatiebron,ontvanger,waarneming,written,heard,seen,else
0,DBNL,1473_Zier_Smit,NaN,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DBNL,1507_Beek_Anon,NaN,65.0,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DBNL,1523_Bosch_OsPe,NaN,251.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DBNL,1526_Zwol_Muld,NaN,13.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DBNL,1535_Gron_Anon,NaN,18.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3,1542_Gent_Anon,"Kroniek van Gent, 1538-1542.",37.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,DBNL,1544_Bred_Anon,NaN,17.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,DBNL,1548_Utre_Mijn,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,DBNL,1551_Kamp_Bred,NaN,475.0,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,DBNL,1553_Utre_Mijn,NaN,7.0,0.0,NaN,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
merged_metadata.head()

cols = ['tokens', 'informatiebron', 'ontvanger', 'waarneming', 'else', 'written', 'heard', 'seen']
merged_metadata[cols] = merged_metadata[cols].fillna(0).applymap(np.int64)

In [36]:
merged_metadata

,Batch,Call_Number,Title,n-scans,Gecontroleerd in project 2,Subcorpus,scans-gelabeld,filename,tokens,informatiebron,ontvanger,waarneming,written,heard,seen,else
0,DBNL,1473_Zier_Smit,NaN,9.0,9.0,NaN,NaN,NaN,0,0,0,0,0,0,0,0
1,DBNL,1507_Beek_Anon,NaN,65.0,37.0,NaN,NaN,NaN,0,0,0,0,0,0,0,0
2,DBNL,1523_Bosch_OsPe,NaN,251.0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
3,DBNL,1526_Zwol_Muld,NaN,13.0,0.0,NaN,NaN,NaN,0,0,0,0,0,0,0,0
4,DBNL,1535_Gron_Anon,NaN,18.0,0.0,NaN,NaN,NaN,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,DBNL,1858_Delf_Stre,NaN,48.0,NaN,NaN,0.0,NaN,0,0,0,0,0,0,0,0
304,DBNL,1859_Nijm_Drui,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
305,DBNL,1862_Leeu_Buma,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
306,DBNL,1876_Nieu_Rybe,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0


In [37]:
merged_metadata.to_csv('merged_metadata_2022.csv')